In [1]:
# fasttext, w2v, GloVe

In [2]:
# use: everything except word and token length

In [42]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import statistics
import random
from sklearn.utils import resample
from nltk.tokenize import sent_tokenize, word_tokenize

In [29]:
# get the df with the posts, tokenized posts, birth years, and binary birth years
df_birth_year = pd.read_csv('data/birth_year.csv')
mil_and_genz = df_birth_year[(1986 < df_birth_year['birth_year']) & (df_birth_year['birth_year'] <= 2006)]
mil_and_genz['binary_birth_year'] = 1
mil_and_genz.loc[(1996 < mil_and_genz['birth_year']) & (mil_and_genz['birth_year'] <= 2006), 'binary_birth_year'] = 0
mil_and_genz = mil_and_genz.reset_index(drop=True)
mil_and_genz

<ipython-input-29-b54abaa40ef8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mil_and_genz['binary_birth_year'] = 1
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,auhtor_ID,post,birth_year,binary_birth_year
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0
...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1


In [33]:
# underample the minority class
undersampled_df = pd.concat([
    resample(mil_and_genz[mil_and_genz['binary_birth_year'] == 1],
             replace=False,
             n_samples=len(mil_and_genz[mil_and_genz['binary_birth_year'] == 0]),
             random_state=42),
    mil_and_genz[mil_and_genz['binary_birth_year'] == 0]
])

# shuffle the dataframe
undersampled_df = undersampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

undersampled_df

,auhtor_ID,post,birth_year,binary_birth_year
23607,t2_3u9vk5dm,"gracia o fortuna, pero lleno de carisma. Su vi...",1992,1
6830,t2_5uxbh2ne,feels like they barely (if at all) gloss over ...,1994,1
14760,t2_sb1vk,"mir, dass das wohl einer der dümmsten und glüc...",1995,1
13566,t2_gx195sh0,Weebs 😖🤢🤮 Is that Madeline from the criminally...,1990,1
5501,t2_5i82y6cm,to go through. And what kind of man wouldn’t w...,1994,1
...,...,...,...,...
25426,t2_4xtxhz2o,- was sent it for free. * In 2008 I got the se...,2000,0
25427,t2_4xtxhz2o,there is precident. I haven't finished the sea...,2000,0
25428,t2_4xtxhz2o,The Next Generation / Enterprise 6. Wonder Wom...,2000,0
25429,t2_4xtxhz2o,episodes from Series 8-9 (23.07%). Both are la...,2000,0


In [36]:
# tokenization
mil_and_genz['post_tokenized'] = mil_and_genz.post.str.findall('\w+|[^\w\s]')
mil_and_genz

,auhtor_ID,post,birth_year,binary_birth_year,post_tokenized
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0,"[Good, on, you, for, being, responsible, !, I,..."
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0,"[must, go, to, the, grocery, store, with, thei..."
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0,"[things, on, her, videos, ,, and, YouTube, too..."
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0,"[their, app, ., There, ', s, also, a, program,..."
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0,"[side, ., If, the, cops, don, ', t, take, your..."
...,...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1,"[clever, political, meme, on, a, sub, about, p..."
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1,"[Kevin, ., Glück, gehabt, ., I, will, try, !, ..."
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1,"[Handy, /, der, Cloud, speichern, und, in, der..."
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1,"[that, ., :, ), ), On, Google, I, cant, find, ..."


In [39]:
# document length
def count_words(tokens):
    return sum(1 for token in tokens if token.isalpha())
mil_and_genz['doc_length'] = mil_and_genz['post_tokenized'].apply(count_words)
mil_and_genz

,auhtor_ID,post,birth_year,binary_birth_year,post_tokenized,doc_length
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0,"[Good, on, you, for, being, responsible, !, I,...",1550
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0,"[must, go, to, the, grocery, store, with, thei...",1560
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0,"[things, on, her, videos, ,, and, YouTube, too...",1542
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0,"[their, app, ., There, ', s, also, a, program,...",1567
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0,"[side, ., If, the, cops, don, ', t, take, your...",1552
...,...,...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1,"[clever, political, meme, on, a, sub, about, p...",1074
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1,"[Kevin, ., Glück, gehabt, ., I, will, try, !, ...",1484
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1,"[Handy, /, der, Cloud, speichern, und, in, der...",1490
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1,"[that, ., :, ), ), On, Google, I, cant, find, ...",1504


In [46]:
# number of sentence
mil_and_genz['nr_sent'] = [len(nltk.sent_tokenize(text)) for text in mil_and_genz['post']]
mil_and_genz

,auhtor_ID,post,birth_year,binary_birth_year,post_tokenized,doc_length,avg_sentence_length,nr_sent
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0,"[Good, on, you, for, being, responsible, !, I,...",1550,18.746835,79
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0,"[must, go, to, the, grocery, store, with, thei...",1560,15.694737,95
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0,"[things, on, her, videos, ,, and, YouTube, too...",1542,17.879518,83
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0,"[their, app, ., There, ', s, also, a, program,...",1567,17.879518,83
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0,"[side, ., If, the, cops, don, ', t, take, your...",1552,16.730337,89
...,...,...,...,...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1,"[clever, political, meme, on, a, sub, about, p...",1074,16.935484,62
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1,"[Kevin, ., Glück, gehabt, ., I, will, try, !, ...",1484,11.926230,122
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1,"[Handy, /, der, Cloud, speichern, und, in, der...",1490,13.407407,108
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1,"[that, ., :, ), ), On, Google, I, cant, find, ...",1504,15.114583,96


In [44]:
# sentence length
mil_and_genz['avg_sentence_length'] = mil_and_genz['post'].apply(lambda text:
    sum(1 for sentence in sent_tokenize(text) for word in word_tokenize(sentence) if word.isalpha()) /
    max(1, len(sent_tokenize(text)))
)
mil_and_genz

,auhtor_ID,post,birth_year,binary_birth_year,post_tokenized,doc_length,avg_sentence_length
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0,"[Good, on, you, for, being, responsible, !, I,...",1550,18.746835
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0,"[must, go, to, the, grocery, store, with, thei...",1560,15.694737
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0,"[things, on, her, videos, ,, and, YouTube, too...",1542,17.879518
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0,"[their, app, ., There, ', s, also, a, program,...",1567,17.879518
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0,"[side, ., If, the, cops, don, ', t, take, your...",1552,16.730337
...,...,...,...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1,"[clever, political, meme, on, a, sub, about, p...",1074,16.935484
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1,"[Kevin, ., Glück, gehabt, ., I, will, try, !, ...",1484,11.926230
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1,"[Handy, /, der, Cloud, speichern, und, in, der...",1490,13.407407
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1,"[that, ., :, ), ), On, Google, I, cant, find, ...",1504,15.114583


In [ ]:
# still need to do stopwords, punctuation, and POS tagging.
# just say for every important word/tag/symbol make diff col and count nr times it is present?